In [1]:
import numpy as np

In [2]:
class WebGraph:
    def __init__(self, n,edges=None):
        self.vertices = n;
        self.matrix = np.zeros((n,n))
        if edges != None:
            self.addEdges(edges)
    
    def addEdge(self,a,b):
        self.matrix[a][b] = 1
    
    def addEdges(self,edges):
        for p in edges:
            self.addEdge(p[0],p[1])
            
    def display(self):
        print("Vertices: "+ str(self.vertices))
        print("Adjacency Matrix:")
        print('x', *list(range(self.vertices)), sep="  ")
        for i in range(self.vertices):
            print(i, self.matrix[i])

In [3]:
web = WebGraph(6)
edges = [(0,1),(0,4),(1,4),(2,0),(3,4),(5,3),(5,2)]
web.addEdges(edges)
web.display()

Vertices: 6
Adjacency Matrix:
x  0  1  2  3  4  5
0 [0. 1. 0. 0. 1. 0.]
1 [0. 0. 0. 0. 1. 0.]
2 [1. 0. 0. 0. 0. 0.]
3 [0. 0. 0. 0. 1. 0.]
4 [0. 0. 0. 0. 0. 0.]
5 [0. 0. 1. 1. 0. 0.]


In [4]:
class PageRank:
    epsilon = 10 ** -6
    max_iter = 10 ** 3
    
    def __init__(self, webGraph, alpha=0.85):
        self.graph = webGraph
        self.alpha = alpha
        self.matrix = self._calc_matrix()
    
    def compute(self,iters=max_iter):
        n = self.graph.vertices
        
        prev_ranks = np.zeros((n,1))
        ranks = np.zeros((n,1)) + (1/n)
        
        for i in range(iters):
            
            prev_ranks = ranks
            ranks = np.dot(self.matrix,ranks)
            
            if self._check_equal(ranks,prev_ranks):
                print("Number of iterations to converge :", str(i))
                break;
        
        return self._display_ranks(ranks)
        
    
    def _calc_matrix(self):
        n = self.graph.vertices
        alpha = self.alpha
        adj_mat = self.graph.matrix.T
        
        e = np.ones((n,1))
        E = (1/n)*np.dot(e,e.T)
        
        outlinks = np.sum(adj_mat,axis=0,keepdims=True)
        temp = np.where(outlinks>0,outlinks,1)
        
        A = adj_mat / temp
        a = np.where(outlinks>0,0,1)
        S = A + (1/n)*np.dot(e,a)

        return alpha*S + (1-alpha)*E
    
    def _check_equal(self,a,b):
        return np.allclose(a,b,atol=PageRank.epsilon)
    
    def _display_ranks(self,ranks):
        print("Ranks:")
        order = [(i,ranks[i][0]) for i in range(self.graph.vertices)]
        print(*order, sep="\n")

In [5]:
p = PageRank(web)
p.compute()

Number of iterations to converge : 12
Ranks:
(0, 0.1727424668224914)
(1, 0.15153453302348358)
(2, 0.11132044585304007)
(3, 0.11132044585304007)
(4, 0.37496271100395856)
(5, 0.07811939744398622)


In [6]:
class HITS:
    epsilon = 10 ** -6
    max_iter = 10 ** 3
    
    def __init__(self,webGraph):
        self.graph = webGraph
        self.A_mat, self.H_mat = self._calc_matrix()
        
    def compute(self,iters=max_iter):
        n = self.graph.vertices
        
        auths = np.zeros((n,1))+(1/n)
        hubs = np.zeros((n,1))+(1/n)
        
        for i in range(iters):
            
            prev_auths = auths
            prev_hubs = hubs
            
            auths = np.dot(self.A_mat,auths)
            hubs = np.dot(self.H_mat,hubs)
            
            auths = auths / np.sum(auths, axis=0)
            hubs = hubs / np.sum(hubs, axis=0)
            
            if self._equal(prev_auths,auths) and self._equal(prev_hubs,hubs):
                print(i)
                break
                
        return self._display_scores(auths,hubs)
    
    def _calc_matrix(self):
        E = self.graph.matrix
        return (np.dot(E.T,E), np.dot(E,E.T))
    
    def _equal(self,a,b):
        return np.allclose(a,b,atol=self.epsilon)
    
    def _display_scores(self,auths,hubs):
        print("Page Score : (Auth, Hub)")
        order = [ [i,(auths[i][0],hubs[i][0])] for i in range(self.graph.vertices)]
        print(*order,sep="\n")

In [7]:
h = HITS(web)
h.compute()

24
Page Score : (Auth, Hub)
[0, (2.7256584815491182e-14, 0.41421334045867064)]
[1, (0.2928926830653403, 0.29289306189625813)]
[2, (9.145792217436314e-07, 1.5966558372254028e-14)]
[3, (9.145792217436314e-07, 0.29289306189625813)]
[4, (0.707105487776189, 0.0)]
[5, (0.0, 5.357487971758285e-07)]
